# Automating monitor rollout
When a large number of monitors needs to be rolled out to a set of similar assets, it is often possible to automate this. In this example, we will consider such a rollout, including the creation of intermediate formula and aggregation tags required in the monitors.

Specifically, we will roll out monitoring on the differential pressure on a set of pumps

## Creating the client
All items will be created for the user as which we log in. Note we can only log in as a local user (not a SSO user) using username and password. It might thus be necessary to create a local user for managing the bulk creation of items.

In [1]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://tm-previous-release301.trendminer.net/"
client_id = "sdktest"
username = "wdaniels"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    username=username,
    password=keyring.get_password(url, username),
    tz="Europe/Brussels"
)

## Retrieving the necessary tags

### By asset
The most straightforward way of creating a monitor per asset would through an asset framework. The cleanest way is to search directly on a specific template:

In [2]:
pumps = client.asset.search(template="Pump")

If necessary, we narrow our search down further:

In [3]:
pumps = client.asset.search(template="Pump", name="Pump1*", description="*water*")

In the (very common) case we do not have any templating on our assets, we can typically still get the assets we need by only searching on asset name:

In [4]:
pumps = client.asset.search(name="Pump1*")

We can then access the tag corresponding to some child attribute for each of our assets:

In [5]:
for pump in pumps:
    pump.get_child_from_name("Flow")
    flow_tag = flow_attribute.tag

### By tagname
If no asset framework is present, we will have to retrieve the correct tags directly from their naming logic. Obviously, this will require tagnames to have some sort of logical structure. In this use case, we will use tagnames directly. Tagnames will be identical except for a numeric unit identifier.

In this example, we will limit ourselves to rolling out monitors only for pump 4.

In [6]:
asset_ids = [4]

## Creating formulas, aggregations, searches and monitors
For each asset, we create a formula with the differential pressure, and then an aggregation taking the moving average of that differential pressure. Then we set up a monitor for when that moving average drops below a given threshold. 

To keep a clear overview, we will collect objects per asset in a dedicated folder, all located within in master folder `Pump Monitoring`.

To keep our script robust, we will first **check if an object already exists**. We only create an new object when it does not (avoiding the error we would otherwise get). If it does exist, we update the existing object to the current definition, which might have changed if we edited our script. This way we can run our script at any time without having to worry about errors/duplicates.

In [15]:
from trendminer_interface.exceptions import ResourceNotFound
import pandas as pd

master_folder = "Pump Monitoring"

for asset_id in asset_ids:

    pump_name = f"Pump {asset_id}"

    # Create new folder if it does not exist yet
    folder = client.folder.from_path(f"{master_folder}/{pump_name}", create_new=True)

    # Intialize formula
    formula = client.formula(
        name=f"TM{asset_id}-HEX-DP.calc",
        description=f"{pump_name} differential pressure",
        formula="out - in",
        mapping={
            "in": client.tag.from_name(f"TM{asset_id}-HEX-PI06201"),
            "out": client.tag.from_name(f"TM{asset_id}-HEX-PI06202"),
        },
        parent=folder,
    )

    # Overwrite existing formula (by identifier), or create new
    try:
        existing_formula = client.formula.from_name(formula.name)
        formula.identifier = existing_formula.identifier
        formula.update()
    except ResourceNotFound:
        formula.save()

    # Initialize aggregation; avg of last 1h
    aggregation = client.aggregation(
        name=f"TM{asset_id}-HEX.DP.avg.1h.calc",
        description=f"{pump_name} differential pressure 1h avg",
        target=formula.tag,
        operator="AVERAGE",
        position="END",
        window=pd.Timedelta(hours=1),
        parent=folder,
    )

    # Overwrite or create aggregation
    try:
        existing_aggregation = client.aggregation.from_name(aggregation.name)
        aggregation.identifier = existing_aggregation.identifier
        aggregation.update()
    except ResourceNotFound:
        aggregation.save()

    # Trigger the indexing so the tags become ready to use asap
    formula.tag.index()
    aggregation.tag.index()

    # Search definition
    search = client.search.value(
        name=f"{pump_name} DP limit",
        description=f"Differential pressure limit reached for pump {pump_name}",
        queries=[
            (aggregation.tag, "<", 0.5),
        ],
        duration=pd.Timedelta(minutes=15),
        parent=folder,
    )

    # Overwrite or create search
    try:
        existing_search = client.search.value.from_name(search.name)
        search.identifier = existing_search.identifier
        search.update()
    except ResourceNotFound:
        search.save()

    # Update monitor
    monitor = search.get_monitor()

    monitor.email.to = "test@trendminer.com,test2@trendminer.com"  # comma separated, no space
    monitor.email.subject = f"{pump_name} low DP!"
    monitor.email.message = f"Maintenance required on {pump_name}"
    monitor.email.enabled = True  # Needs to be turned on explicitly

    monitor.context.context_type = client.context.type.from_key("ANOMALY")  # unique key, not name!
    monitor.context.component = aggregation.tag  # attaching to aggregation tag itself
    monitor.context.description = "Maintenance required"
    monitor.context.fields = {"Reason": "Low DP"}  # use unique field keys
    monitor.context.keywords = ["maintenance"]
    monitor.context.enabled = True  # Needs to be turned on explicitly

    monitor.enabled = True  # Global activation of monitor required
    monitor.update()
